# LDA with Variational Bayes


## train & test データロード


In [1]:
def load_train_test():
    """
    @return train list 学習用の文書集合
    @return test list テスト用の文書集合
    """
    read_dir = './data/ldcourpas/'
    train_doc_name = 'train_doclist.list'
    test_doc_name = 'test_doclist.list'
    
    with open(read_dir + train_doc_name, mode='rb') as f:
        train = pickle.load(f)
    with open(read_dir + test_doc_name, mode='rb') as f:
        test = pickle.load(f)
    
    return train, test

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pickle

train, test = load_train_test()
analyzer = lambda words: words
vect = CountVectorizer(max_features=10000, min_df=.01, max_df=.40, analyzer=analyzer)
X = vect.fit_transform(train)
id2word = {v: k for k, v in vect.vocabulary_.items()}

## LDA


In [2]:
import scipy.special as special


def _dirichlet_expectation_1d(arr):
    """
    calcurate (E[log(vars)]), vars = Dir(vars|arr)
    """
    sum_arr = arr.sum()
    return special.psi(arr) - special.psi(sum_arr)

def _dirichlet_expectation_2d(arr):
    """
    calcurate E[log(vars)], vars ~ Dir(vars|arr)
    """
    sum_axis1 = arr.sum(axis=1).reshape(-1, 1)
    return special.psi(arr) - special.psi(sum_axis1)
    

In [ ]:
import logging
import time
import scipy.special as special
import scipy.stats as stats
from joblib import Parallel, delayed, cpu_count

EPS = np.finfo(np.float).eps


def _get_n_jobs(n_jobs):
    if n_jobs < 0:
        return max(cpu_count() - 1, 1)
    elif n_jobs == 0:
        ValueError('n_jobs == 0 doesn\'t meaning')
    else:
        return n_jobs
    

def gen_slices(length, n):
    """
    divide idx[0:length] into n
    """
    idx = np.arange(length)
    nums = [(length + i) // n for i in range(n)]
    
    start = 0
    for num in nums:
        end = start + num
        yield idx[start: end]
        start = end


def mean_change(arr1, arr2):
    """
    calculate mean abs difference between two arrays.
    """
    size = arr1.shape[0]
    return np.abs(arr1 - arr2).sum() / size

def _init_doctopic(n_features, n_topics, alpha, nd):
    phi_d = np.ones((n_features, n_topics)) / n_topics
    gamma_d = alpha + nd / n_topics
    return phi_d, gamma_d

def _update_doctopic_ditribution(X, max_iter, alpha,
                                 exp_topic_word_distr,
                                 mean_change_tol,
                                 random_state,
                                 cal_suff_stats):
    n_docs, n_features = X.shape
    n_topics = exp_topic_word_distr.shape[0]
    
    indices = X.indices
    indptr = X.indptr
    data = X.data
    
    # In the literature, doc_topic is called 'γdk'
    doc_topic_distr =  doc_topic_distr = random_state.gamma(100.,
                                                            0.01,
                                                            (n_docs, n_topics))
    exp_doc_topic_ = np.exp(_dirichlet_expectation_2d(doc_topic_distr))
    
    suff_stats = np.zeros(exp_topic_word_distr.shape) if cal_suff_stats else None
    
    for d in range(n_docs):
        ids = indices[indptr[d]:indptr[d + 1]]
        ndw = data[indptr[d]:indptr[d + 1]]
        
        doc_topic_d = doc_topic_distr[d]
        exp_doc_topic_d = exp_doc_topic_[d]
        exp_topic_word_d = exp_topic_word_distr[:, ids]
        
        for _ in range(max_iter):
            last_d = doc_topic_d
            
            norm_phi = np.dot(exp_doc_topic_d, exp_topic_word_d)
            
            doc_topic_d = alpha + (exp_doc_topic_d *\
                           np.dot(ndw/norm_phi, exp_topic_word_d.T))
            
            exp_doc_topic_d = np.exp(_dirichlet_expectation_1d(doc_topic_d))
            
            # check convergence
            if mean_change(doc_topic_d, last_d) < mean_change_tol:
                break
        
        doc_topic_distr[d] = doc_topic_d
        
        if cal_suff_stats:
            norm_phi = np.dot(exp_doc_topic_d, exp_topic_word_d)
            suff_stats[:, ids] += ndw/norm_phi\
                                    * exp_topic_word_d\
                                    * exp_doc_topic_d.reshape(-1, 1)
    
    return doc_topic_distr, suff_stats
    

class LDA(object):
    PRINT_EVERY = 20
    def __init__(self, n_topics=10, outer_iter=10, max_update_iter=100,
                 mean_change_tol=1e-3, n_jobs=-1, verbose=0,
                 logger=None):
        self.n_topics = n_topics
        self.outer_iter = outer_iter
        self.max_update_iter = max_update_iter
        self.mean_change_tol = mean_change_tol
        self.n_jobs = n_jobs
        self.verbose = verbose
        self.logger = logger
        self.random_state = np.random.mtrand._rand
    
    def _initialize(self, X):
        n_docs, n_features = X.shape
        
        self.alpha = np.ones(self.n_topics) * 0.1
        self.eta = 10 / n_features
        
        init_gamma = 100.
        init_var = 1. / init_gamma
        
        # In the literature, 'lambda'
        self.topic_ = self.random_state.gamma(init_gamma,
                                             init_var,
                                             (self.n_topics, n_features))
        
        # In the literature, 'exp(E[log(beta))])
        self.exp_topic_word_distr = np.exp(
            _dirichlet_expectation_2d(self.topic_))
        
        indices = X.indices
        indptr = X.indptr
        data = X.data
        self.nd = np.zeros(n_docs)
        for d in range(n_docs):
            ndw = data[indptr[d]:indptr[d + 1]]
            self.nd[d] = ndw.sum()
        
    def _update_dirichlet_param(self, doc_topic):
        """
        update alpha

        @param ave_ndz ndarray ndzのサンプル平均
        """
        e_ndk = doc_topic - self.alpha
        n_docs = e_ndk.shape[0]
        sum_alpha = self.alpha.sum()
        
        numes = (special.psi(e_ndk + self.alpha).sum(axis=0)\
                    - n_docs*special.psi(self.alpha))*self.alpha
        
        denom = special.psi(self.nd + sum_alpha).sum()\
                        - n_docs*special.psi(sum_alpha)
        
        self.alpha = numes / denom
        
    def _e_step(self, X, parallel=None):
        """
        e-step in EM.
        """
        n_jobs = _get_n_jobs(self.n_jobs)
        if parallel is None:
            parallel = Parallel(n_jobs=n_jobs, verbose=max(0,
                                                           self.verbose-1))
        
        cal_suff_stats = True
        results = parallel(
            delayed(_update_doctopic_ditribution)(X[idx_slice, :],
                                                  self.max_update_iter,
                                                  self.alpha,
                                                  self.exp_topic_word_distr,
                                                  self.mean_change_tol,
                                                  self.random_state,
                                                  cal_suff_stats)
            for idx_slice in gen_slices(X.shape[0], n_jobs))
        
        doc_topics, suff_stats_list = zip(*results)
        doc_topic = np.vstack(doc_topics)
        
        if cal_suff_stats:
            suff_stats = np.zeros(self.exp_topic_word_distr.shape)
            for stats in suff_stats_list:
                suff_stats += stats
        else:
            suff_stats = None
        
        return doc_topic, suff_stats
        
    def _em_step(self, X):
        """
        EM update for 1 iteration.
        
        """
        doc_topic, suff_stats = self._e_step(X)
        
        self.topic_ = self.eta + suff_stats
        self.exp_topic_word_distr = np.exp(
            _dirichlet_expectation_2d(self.topic_))
        
        return
        # update α
        self._update_dirichlet_param(doc_topic)
        
    def fit(self, X):
        n_docs, n_features = X.shape
        
        self._initialize(X)
        
        if self.logger:
            self.logger.info('train start!')
        
        for out_s in range(self.outer_iter):
            start = time.time()
            
            self._em_step(X)
            
            elapsed_time = time.time() - start
            if self.logger:
                self.logger.info('total {} iter finish!'.format(out_s+1))
                self.logger.info('elapsed time: {:.3f} [sec]'\
                                 .format(elapsed_time))
    
    def print_topn_words(self, n, id2word):
        index = np.arange(n) + 1
        df = pd.DataFrame(data=[], index=index)
        for k in range(self.n_topics):
            idx_descend = self.topic_[k].argsort()[::-1]
            top_n = [id2word[idx] for idx in idx_descend[:n]]
            df['topic{}'.format(k)] = top_n
        
        display(df)
        

def main():
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger('LDA')
    lda = LDA(outer_iter=25, max_update_iter=100, logger=logger)
    lda.fit(X)
    lda.print_topn_words(n=10, id2word=id2word)
    
main()

INFO:LDA:train start!
